In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import plotly.express as px

In [ ]:
MODELS_PATH = Path("/data/toulouse/bicycle/notebooks/experiments/bottleneck/data/models")
PLOTS_PATH = Path("/data/toulouse/bicycle/notebooks/experiments/bottleneck/data/plots")
ANALYSIS_PATH = Path("/data/toulouse/bicycle/notebooks/experiments/bottleneck/data/analysis")
exclude=["test_run_00013", "test_run_00014", "figures"]

In [ ]:
# run_id        profile scale compile full dynamic mode   worker_num  name

parameters = pd.read_csv(ANALYSIS_PATH/"manual_params.csv").set_index("run_id", drop=True)
keys = pd.Series([
    "test_run_00058",
    "test_run_00057",
    ])
parameters = parameters.loc[keys]

#
#parameters.loc["test_run_00053"] = manual_params.loc["test_run_00053"]
#keys.loc[len(keys)] = "test_run_00053"

In [ ]:
parameters

## Summary profiles

In [ ]:
training_profiles = dict()
for dir in ANALYSIS_PATH.iterdir():
    if str(dir.name) in exclude:
        continue
    if dir.is_dir() and dir.name in keys.to_list():
        training_profiles[str(dir.name)] = pd.read_csv(dir/"training_profile.csv")


In [ ]:
# calculate filtered profiled time
times = list()
for key in keys:
    print(key)
    times.append(training_profiles[key]["Time"].sum())
parameters["Profile_Time"] = times

In [ ]:
title="Comparison of profiled time with different trainer precisions"
fig = px.bar(parameters.sort_values("Profile_Time").reset_index(),
       x="trainer_precision",
       y="Profile_Time",
       text="Profile_Time",
       title=title,
       color="trainer_precision",
       color_continuous_scale="sunset"
       )
fig.show()
fig.write_image(ANALYSIS_PATH/"figures"/f"{title}.pdf", scale=6)

In [ ]:
title="Comparison of total training time with different trainer precisions"
fig = px.bar(parameters.sort_values("training_time").reset_index(),
       x="trainer_precision",
       y="training_time",
       text="training_time",
       title=title,
       color="trainer_precision",
       color_continuous_scale="sunset",
       )
fig.show()
fig.write_image(ANALYSIS_PATH/"figures"/f"{title}.pdf", scale=6)

In [ ]:
aggregator = {
    "Class": lambda x: x.iloc[0],
    "Function": lambda x: x.iloc[0],
    "Class_Function_etc": lambda x: x.iloc[0],
    "Class_Function": lambda x: x.iloc[0],
    "Summary_index": lambda x: x.iloc[0],
    "filename_lineno(function)": lambda x: x.iloc[0],
    "is_callback": lambda x: x.iloc[0],
    "Call_num": "sum",
    "Primitive_Call_num": "sum",
    "Time": "sum",
    "ncalls": "sum",
    "tot_time": "sum",
    "tot_percall": "mean",
    "cum_time": "sum",
    "cum_percall": "mean",
}

In [ ]:
full_profiles = dict()
for dir in ANALYSIS_PATH.iterdir():
    if str(dir.name) in exclude:
        continue
    if dir.is_dir() and dir.name in keys.to_list():
        df = pd.read_csv(dir.joinpath("full_training_profile.csv")).drop(columns = ["Rank"])
        df = df.groupby(["Class_Function_etc"], as_index=False, ).agg(aggregator).reset_index()
        df["in_model"] = df["Class_Function_etc"].apply(lambda x: "model.py" in str(x).casefold())
        df["in_bicycle"] = df["Class_Function_etc"].apply(lambda x: "bicycle" in str(x).casefold())
        df = df.sort_values("filename_lineno(function)").set_index(pd.Index(np.arange(len(df))),drop=True)
        #numericals = ["Call_num","Primitive_Call_num","Time","ncalls","tot_time","tot_percall","cum_time","cum_percall"]
        #df[numericals] = df[numericals]/sum(df[numericals], axis=0)
        full_profiles[str(dir.name)] = df

In [ ]:
print(full_profiles.keys())

In [ ]:
metrics = ["ncalls","tot_time","tot_percall","cum_time","cum_percall"]

if parameters.index.name != "run_id":
    parameters=parameters.set_index("run_id")
condition="in_model"
top_df = pd.DataFrame(columns=df.columns.append(parameters.columns))
for n, df in full_profiles.items():
        data = df.query(condition)#.sort_values(metric, ignore_index=True, ascending = False).iloc[:20]
        for _, row in data.iterrows():
            top_df.loc[len(top_df)] = np.concatenate([row, parameters.loc[n]], axis=0)

In [ ]:
aggregator["name"]= lambda x: x.iloc[0]
for metric in metrics:
    title = f"Trainer Precsion: {metric} of functions with {condition}"
    
    data = top_df.groupby(["loader_workers", "Class_Function_etc"],).agg(aggregator)
    fig = px.bar(data.sort_values(metric).sort_values("loader_workers"),
    x = "filename_lineno(function)",
    y = metric,
    color= "trainer_precision",
#    log_y = True,
    text="Function",
    title=title,
    barmode="group",
    )
    fig.show()
    #fig.write_image(ANALYSIS_PATH/"figures"/f"{title}.pdf")

In [ ]:
top_df.loc[top_df["filename_lineno(function)"] == "model.py:828(split_samples)"]